In [1]:
import gradio as gr
import random
import os
import dotenv
import json
from openai import OpenAI


In [2]:
%load_ext dotenv
%dotenv

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')

# Model choices: default + 2 additional frontier OpenAI models
MODEL_CHOICES = ["gpt-4.1-mini", "gpt-4o", "gpt-4-turbo"]
openai = OpenAI()


In [4]:
system_message = """
You are a very helpful assistant that always ensures you find a solution or at least a good suggestion to your manager's challenges.
"""

def getMessage(message, history):
    sys = [{"role": "system", "content": system_message}]
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    msg = [{"role": "user", "content": message}  ]
    return sys + history + msg


In [5]:
# 10 employees: name -> email (names and emails not directly matched)
employees = {
    "jp": "j.park@company.com",
    "robert": "r.martinez@company.com",
    "alice": "a.wong@company.com",
    "bob": "b.smith@company.com",
    "carol": "c.nelson@company.com",
    "eve": "e.kovacs@company.com",
    "frank": "f.garcia@company.com",
    "sarah": "s.johnson@company.com",
    "mike": "m.chen@company.com",
    "nina": "n.patel@company.com",
}


def get_email(name):
    if not name:
        return "No name provided."
    key = name.strip().lower()
    if key in employees:
        return employees[key]
    return "I don't have that employee on file."


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_employee_email_by_name",
            "description": "Get the email address of an employee given their name",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The employee's name, e.g. Dola, Emmanuel",
                    },
                },
                "required": ["name"],
            },
        },
    },
]

In [6]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_employee_email_by_name":
            arguments = json.loads(tool_call.function.arguments)
            name = arguments.get("name")
            email = get_email(name)
            responses.append({
                "role": "tool",
                "content": email,
                "tool_call_id": tool_call.id
            })
    return responses

In [7]:
def _accumulate_tool_calls_from_stream(stream):
    """Consume stream and accumulate content + tool_calls; return (content, tool_calls_list, finish_reason)."""
    content = ""
    tool_calls_by_index = {}
    finish_reason = None
    for chunk in stream:
        delta = chunk.choices[0].delta if chunk.choices else None
        if not delta:
            continue
        if getattr(delta, "content", None):
            content += delta.content or ""
        if getattr(delta, "tool_calls", None):
            for tc in delta.tool_calls:
                idx = tc.index
                if idx not in tool_calls_by_index:
                    tool_calls_by_index[idx] = {"id": "", "type": "function", "function": {"name": "", "arguments": ""}}
                if tc.id:
                    tool_calls_by_index[idx]["id"] = tc.id
                if getattr(tc.function, "name", None):
                    tool_calls_by_index[idx]["function"]["name"] = tc.function.name
                if getattr(tc.function, "arguments", None):
                    tool_calls_by_index[idx]["function"]["arguments"] += tc.function.arguments or ""
        if chunk.choices and chunk.choices[0].finish_reason:
            finish_reason = chunk.choices[0].finish_reason
    tool_calls_list = [tool_calls_by_index[i] for i in sorted(tool_calls_by_index)] if tool_calls_by_index else None
    return content, tool_calls_list, finish_reason


def chat(message, history, model):
    prompt = getMessage(message, history)
    accumulated = ""

    while True:
        stream = openai.chat.completions.create(
            model=model, messages=prompt, tools=tools, stream=True
        )
        content, tool_calls_list, finish_reason = _accumulate_tool_calls_from_stream(stream)
        accumulated += content
        yield accumulated

        if finish_reason != "tool_calls" or not tool_calls_list:
            break

        # Build assistant message for tool round (dict format for API)
        assistant_msg = {
            "role": "assistant",
            "content": content or None,
            "tool_calls": [
                {
                    "id": tc["id"],
                    "type": "function",
                    "function": {"name": tc["function"]["name"], "arguments": tc["function"]["arguments"]},
                }
                for tc in tool_calls_list
            ],
        }
        prompt.append(assistant_msg)
        tool_responses = []
        for tc in tool_calls_list:
            fn_name = tc["function"]["name"]
            args_str = tc["function"]["arguments"]
            if fn_name == "get_employee_email_by_name":
                args = json.loads(args_str) if args_str else {}
                email = get_email(args.get("name", ""))
                tool_responses.append({
                    "role": "tool",
                    "content": email,
                    "tool_call_id": tc["id"],
                })
        prompt.extend(tool_responses)

    # No further yields; Gradio uses the last yielded value


In [8]:
model_dropdown = gr.Dropdown(
    choices=MODEL_CHOICES,
    value=MODEL_CHOICES[0],
    label="Model",
)

gr.ChatInterface(
    fn=chat,
    type="messages",
    additional_inputs=[model_dropdown],
).launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
